In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
current_dir = '/content/drive/MyDrive/tutorial/Regional-CNN-LSTM'
print(current_dir)
model_path = os.path.join(current_dir,'model')
if os.path.exists(model_path) == False:
  os.makedirs(model_path)

embedding_path = os.path.join(current_dir,'word2vec')
if os.path.exists(embedding_path) == False:
  os.makedirs(embedding_path)

/content/drive/MyDrive/tutorial/cnn-bilstm


斷句＆斷詞

In [ ]:
# 載入繁體中文jieba斷詞＆斷句需使用的trankit套件
!pip install git+https://github.com/APCLab/jieba-tw.git

  Cloning https://github.com/APCLab/jieba-tw.git to /tmp/pip-req-build-to4aiq46
  Running command git clone --filter=blob:none --quiet https://github.com/APCLab/jieba-tw.git /tmp/pip-req-build-to4aiq46
  Resolved https://github.com/APCLab/jieba-tw.git to commit 2bd4c0913cdadc865879b21a00c78e28ef1ba2c5
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import jieba
from tqdm import tqdm
import re

training_data = pd.read_csv(os.path.join(current_dir,'CVAT_all.csv'),sep=',')
validation_data = pd.read_csv(os.path.join(current_dir,'DSAMST-ValidationSet_ans.csv'),sep=',')

def data_load(data):
  x, y_valence, y_arousal = [], [], []
  with open(os.path.join(embedding_path,"segmented_corpus.txt"), "w", encoding="utf-8") as f:
    for i in tqdm(range(len(data))):
        segmented_line = " ".join(jieba.cut(data.iloc[i]['Text']))  # 使用空格分隔詞
        f.write(segmented_line + "\n")
        y_valence.append(float(data.iloc[i]['Valence']))
        y_arousal.append(float(data.iloc[i]['Arousal']))

        sentences = re.split(r'[。！？;；]', segmented_line)  # 用標點符號拆分
        sentences = [s.strip() for s in sentences if s.strip()]
        x.append([segmented_line,sentences])
  return x, y_valence, y_arousal

x_train, y_train_valence, y_train_arousal = data_load(training_data)
x_valid, y_valid_valence, y_valid_arousal = data_load(validation_data)

Loading pretrained XLM-Roberta, this may take a while...


/usr/local/lib/python3.11/dist-packages/adapters/composition.py:186: FutureWarning: Passing list objects for adapter activation is deprecated. Please use Stack or Fuse explicitly.
  warnings.warn(


Loading tokenizer for traditional-chinese
Loading tagger for traditional-chinese
Loading lemmatizer for traditional-chinese
Loading NER tagger for traditional-chinese
Active language: traditional-chinese
Turning on auto mode for ['traditional-chinese'] ...
Trankit is in auto mode!


  0%|          | 0/2969 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 0.406 seconds.
DEBUG:jieba:Loading model cost 0.406 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.
100%|██████████| 994/994 [03:12<00:00,  5.17it/s]


# 詞嵌入向量訓練 -> 近期版本更新，需處理相容問題

In [ ]:
!pip install gensim

from gensim.models import word2vec

sentences = word2vec.LineSentence(os.path.join(embedding_path,"segmented_corpus.txt"))
model = word2vec.Word2Vec(sentences, vector_size=300,min_count=1,epochs=10)

#保存模型，供日後使用
model.wv.save_word2vec_format(os.path.join(embedding_path,'word2vec_embedding.txt'), binary=False)

資料處理

In [ ]:
from collections import defaultdict
import ast

def build_data_train_valid(x_train, y_train_valence, y_train_arousal, x_valid, y_valid_valence, y_valid_arousal):
    revs = []
    vocab = defaultdict(float)

    for i in range(len(x_train)):
        orig_rev, region_rev = x_train[i][0],x_train[i][1]
        words = set(orig_rev.split(' '))
        for word in words:
            vocab[word] += 1
        datum = {
            'valence': y_train_valence[i],
            'arousal': y_train_arousal[i],
            'text': orig_rev,
            'region_text': region_rev,
            'num_regions': len(region_rev),
            'num_words': len(orig_rev.split(' ')),
            'split': 'train'
        }
        revs.append(datum)
    print('Train Done')

    for i in range(len(x_valid)):
        orig_rev, region_rev = x_valid[i][0],x_valid[i][1]

        words = set(orig_rev.split(' '))
        for word in words:
            vocab[word] += 1
        datum = {
            'valence': y_valid_valence[i],
            'arousal': y_valid_arousal[i],
            'text': orig_rev,
            'region_text': region_rev,
            'num_regions': len(region_rev),
            'num_words': len(orig_rev.split(' ')),
            'split': 'valid'
        }
        revs.append(datum)
    print('Validation Done')

    return revs, vocab

In [ ]:
revs, vocab = build_data_train_valid(x_train, y_train_valence, y_train_arousal,  \
                                            x_valid, y_valid_valence, y_valid_arousal)

Train Done
他 先 在 家庭 醫生 的 協助 下 就診 , 發現 左側 胸腔 積液 。
描述 事件 具體 發生 了 什麼 事 根據 病人 的 報告 , 他 三 天前 開始 出現 呼吸 急促 , 並 伴隨 輕微 的 左 胸痛 和 乾咳 。
可以 怎麼 做 我 覺得 真的 可以 找 他們 聊聊 家 中 狀況 , 或 多 聽 多 看 病人 在 字裡行間 透露出 什麼 訊息 , 不過 我 想 家人 真的 是 不 想 放棄 才 會 答應 心臟 移植 , 但 也 需要 為家 中 其 他 健康 的 家人 著想 , 如果 弟弟 真的 回天乏術 , 放棄 也 何嘗 不 是 一 個 選擇 , 但 看到 病人 正 努力 回復 LVEF , 如果 慢慢 得到 好轉 , 讓 家屬 看見 希望 , 進行 心臟 移植 的 意願 應該 也 會 隨之 提高 , 因此 醫療 方盡 全力 做到 最好 , 若 病人 的 生命力 旺盛 到 真的 有機 可行 , 最終 可能 也 會 有 一 個 好 結果 。
當下 很 想 問問 他們 的 家 裡 狀況 跟 家屬 心中 在 想 什麼 , 但 因為 要 跟 著 查房 且 家屬 想多 陪陪 病人 , 所以 就 沒 多問 , 這 種 較 私人 的 話 還是 先 交由 經驗 豐富 、 講話 比較 有 藝術 的 護理師 問 好了 。
反應 為何 當下 我 看見 他們 家屬 間 有 說不上 的 尷尬 感 , 但 卻 也 看見 他們 不斷 陪 著 他 聊天 講話 。
但 最後 又 聽說 他們 家 的 經濟 狀況 其實 不 太 好 , 病患 本人 又 是 做工 的 存款 也 沒有 很多 , 也 無 妻兒 , 因此 重擔 落在 哥哥 身 上 , 自費 albumin 時一袋 1700 元 * 6 都 非常 猶豫 , 矛盾 心情 全 透漏 在 他們 臉 上 。
描述 事件 這 個 病患 ( 57 歲 男性 ) 從 剛 入院 就是 我 的 primarycare , 從 第一天 看 他 接上 ECMO 並 做完 primaryPCI 就 來到 CCU , 從 一 開始 的 E1VeM1 到 現在 第 九天 進步 到 E2VeM3 , 從 一 開始 的 LVEF 個位數 到 現在 的 16 , 固然 有 慢慢 在 進步 , hemodynamic 還是 慘不忍睹 , 因此 詢

In [ ]:
import numpy as np
import pandas as pd
max_l = np.max(pd.DataFrame(revs)['num_words'])
mean_l = np.mean(pd.DataFrame(revs)['num_words'])
print('data loaded!')
print('number of sentences: ' + str(len(revs)))
print('vocab size: ' + str(len(vocab)))
print('max sentence length: ' + str(max_l))
print('mean sentence length: ' + str(mean_l))

data loaded!
number of sentences: 3963
vocab size: 20316
max sentence length: 226
mean sentence length: 38.40903356043401


In [ ]:
def get_W(word_vecs, embedding_dim=300):
    """
    Get word matrix. W[i] is the vector for word indexed by i
    """
    vocab_size = len(word_vecs)
    word_idx_map = dict()
    # position 0 was not used
    W = np.zeros(shape=(vocab_size+2, embedding_dim), dtype=np.float32)
    W[0] = np.zeros((embedding_dim, ))
    W[1] = np.random.uniform(-0.25, 0.25, embedding_dim)

    i = 2
    for word in word_vecs:
        try:
            W[i] = word_vecs[word]
            word_idx_map[word] = i
            i = i + 1
        except:
            continue
    return W, word_idx_map

In [ ]:
import pickle
import os

embeddig_path = os.path.join(embedding_path,'word2vec_embedding.txt')
word2vec = pd.read_csv(embeddig_path, sep=" ", quoting=3, header=None, index_col=0, skiprows=1)
embeddings_index = {key: val.values for key, val in word2vec.T.items()}
w2v = embeddings_index

print('word embeddings loaded!')
print('num words in embeddings: ' + str(len(w2v)))

W, word_idx_map = get_W(w2v)
print('extracted index from embeddings! ')

pickle_file = os.path.join(current_dir, f'tutorial.pickle3')
pickle.dump([revs, W, word_idx_map, vocab, max_l], open(pickle_file, 'wb'))

word embeddings loaded!
num words in embeddings: 5122
extracted index from embeddings! 


Reginal CNN-LSTM 訓練

In [ ]:
#載入所需要的函式
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Convolution1D, MaxPooling1D, Dense, Flatten, TimeDistributed
from keras.layers import Embedding, LSTM, Bidirectional, Input
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr

In [ ]:
pickle_file = os.path.join(current_dir,'tutorial.pickle3')
revs, W, word_idx_map, vocab, max_len = pickle.load(open(pickle_file, 'rb'))
id2emb = W

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.test.is_built_with_cuda())

option = 'arousal'

batch_size = 32
epochs = 10
hidden_dim = 256

kernel_size = 3
nb_filter = 256

max_l = 512
max_r = 50

Num GPUs Available:  1
True


In [ ]:
def make_idx_data(revs, word_idx_map, maxlen, max_region):
    """
    Transforms sentences into a 2-d matrix.
    """
    X_train, X_valid, y_train, y_valid = [], [], [], []
    for rev in revs:
        sent = np.zeros((max_region, maxlen))
        region_text = rev['region_text']
        for i, region in enumerate(region_text):
            if i >= max_region:
                continue
            for j, word in enumerate(region.split()):
                if j == maxlen:
                    break
                if word in word_idx_map:
                    sent[i, j] = word_idx_map[word]
                else:
                    sent[i, j] = 1
        y = rev[option]

        if rev['split'] == 'train':
            X_train.append(sent)
            y_train.append(y)
        elif rev['split'] == 'valid':
            X_valid.append(sent)
            y_valid.append(y)

    X_train = np.array(X_train, dtype='int')
    X_valid = np.array(X_valid, dtype='int')
    y_train = np.array(y_train)
    y_valid = np.array(y_valid)

    return [X_train, X_valid, y_train, y_valid]

In [ ]:
X_train, X_valid, y_train, y_valid = make_idx_data(
                    revs, word_idx_map, maxlen=max_l, max_region=max_r)

n_train_sample = X_train.shape[0]
print("n_train_sample [n_train_sample]: %d" % n_train_sample)

n_valid_sample = X_valid.shape[0]
print("n_valid_sample [n_valid_sample]: %d" % n_valid_sample)

len_region = X_train.shape[1]     # 200
print("len_region [len_region]: %d" % len_region)

len_sentence = X_train.shape[2]     # 200
print("len_sentence [len_sentence]: %d" % len_sentence)

max_features = W.shape[0]
print("num of word vector [max_features]: %d" % max_features)

num_features = W.shape[1]               # 400
print("dimension num of word vector [num_features]: %d" % num_features)

n_train_sample [n_train_sample]: 2969
n_valid_sample [n_valid_sample]: 994
len_region [len_region]: 50
len_sentence [len_sentence]: 512
num of word vector [max_features]: 5124
dimension num of word vector [num_features]: 300


In [ ]:
# 建立 Keras Model
sentence_input = Input(shape=(max_l,), dtype='int32')
embedded_sequences = Embedding(input_dim=max_features, output_dim=num_features, input_length=max_l, weights=[W], trainable=False) (sentence_input)
l_convolution = Convolution1D(filters=nb_filter,
                        kernel_size=kernel_size,
                        padding='valid',
                        activation='relu',
                        strides=1
                        ) (embedded_sequences)

l_maxpooling = MaxPooling1D(pool_size=2) (l_convolution)
l_cnn = Flatten() (l_maxpooling)

sentEncoder = Model(sentence_input, l_cnn)

review_input = Input(shape=(max_r, max_l), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(LSTM(hidden_dim))(review_encoder)
preds = Dense(1, activation='linear') (l_lstm_sent)
model = Model(review_input, preds)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 50, 512)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 50, 65280)           │       1,767,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 512)                 │     134,219,776 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 135,988,145 (518.75 MB)

 Trainable params: 134,450,945 (512.89 MB)

 Non-trainable params: 1,537,200 (5.86 MB)

In [ ]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)
y_pred = model.predict(X_valid, batch_size=batch_size).flatten()

predict_file = open(os.path.join(model_path,f'{option}_predict.txt'),'w')
for y,pred in zip(y_valid, y_pred):
  print(y,pred,file=predict_file)
predict_file.close()

In [ ]:
def evaluate(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    pr = pearsonr(y_true, y_pred)[0]

    print('Evaluation length: %d' % len(y_pred))
    print('MAE: %.3f' % (mae))
    print('Pearsonr: %.3f' % (pr))
    return mae,pr

In [ ]:
mae,pr = evaluate(y_valid, y_pred)

score = open(os.path.join(model_path,f'{option}_score.txt'),'w')
score.write('MAE: %.3f' % (mae)+'\n')
score.write('Pearsonr: %.3f' % (pr)+'\n')
score.close()


Evaluation length: 994
MAE: 1.356
Pearsonr: 0.112
